In [ ]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import keras
from keras import metrics
from keras import regularizers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import Adam, RMSprop
from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from keras.utils import plot_model
from keras.models import load_model
import matplotlib.pyplot as plt



df = pd.read_excel(r'Copy of Extraction PT4 14x59 din 2016 pana in prezent - analysis S1928 (003) (3).xlsx', sheet_name='TL4 -40 Nm' )
dataset = df.values
X = dataset[1:,23:42]
Y = dataset[1:,14]
X = X.astype('float')
Y = Y.astype('float')


d1 = pd.DataFrame(X)
d2 = pd.DataFrame(Y)
d1.drop([1,2,3,5,6,7,8,9,10,11,13,14,15,17,18], axis=1, inplace=True)
d2.replace(0.  , np.nan, inplace=True)
d2.replace(-1. , np.nan, inplace=True)
d2.replace(1.  , np.nan, inplace=True)
d2.replace(2.  , np.nan, inplace=True)
d2.replace(3.  , np.nan, inplace=True)
d2.replace(4.  , np.nan, inplace=True)
d2.replace(5.  , np.nan, inplace=True)
d2.replace(6.  , np.nan, inplace=True)

d1[7] = d2[0]
d1.dropna(inplace=True)
# d1.isnull().any() # Verifica daca avem vre-o valoare de tip NaN

X = d1.values[0:,:-1]
Y = d1.values[0:, -1:]


min_max_scaler = preprocessing.MinMaxScaler()
X_scale = min_max_scaler.fit_transform(X)
Y_scaler = preprocessing.MinMaxScaler()
Y_scale = Y_scaler.fit_transform(Y)




X_train, X_test, Y_train, Y_test = train_test_split(X_scale, Y_scale, test_size=0.2)
X_train.shape, Y_train.shape, X_test.shape, Y_test.shape




def basic_model_3(x):
     # create model
    model = Sequential()
    model.add(Dense(x, input_dim=4, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(x, input_dim=4, kernel_initializer='normal', activation='relu'))
    model.add(Dense(x, input_dim=2, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

def error(x):
    emax = 1.
    emin = 1.
    for i in range(x):
        p = model.predict(X_train[i].reshape(1, 4))
        #print(p, Y_train[i], p/float(Y_train[i]))
        pp = Y_scaler.inverse_transform(p)
        py = Y_scaler.inverse_transform(Y_train[i].reshape(1,-1))
        r = p/float(Y_train[i])
        if r > emax and py > 7.:
            emax = r
        if r < emin and py > 7.:
            emin = r
        print(pp,py, r)
    return emin , emax;

rmin = 10
rmax = 100

for x4 in range(rmin,rmax):
                dif=3.0
                model = basic_model_3(x4)
                epochs = 10000
                batch_size = 128
                #print('Epochs: ', epochs)
                #print('Batch size: ', batch_size)
                keras_callbacks = [
                # ModelCheckpoint('/tmp/keras_checkpoints/model.{epoch:02d}-{val_loss:.2f}.hdf5', monitor='val_loss', save_best_only=True, verbose=2)
                # ModelCheckpoint('/tmp/keras_checkpoints/model.{epoch:02d}.hdf5', monitor='val_loss', save_best_only=True, verbose=0)
                # TensorBoard(log_dir='/tmp/keras_logs/model_3', histogram_freq=0, write_graph=True, write_images=True, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None),
                EarlyStopping(monitor='val_mean_absolute_error', patience=20, verbose=0)
                ]



                history = model.fit(X_train, Y_train,
                batch_size=batch_size,
                epochs=epochs,
                shuffle=True,
                verbose=0, # Change it to 2, if wished to observe execution
                validation_data=(X_test, Y_test),
                callbacks=keras_callbacks)
                
                train_score = model.evaluate(X_train, Y_train, verbose=0)
                valid_score = model.evaluate(X_test, Y_test, verbose=0)
            
                emn , ema = error(50)
                print('errmin,errmax,x4:',emn,ema , x4)

Using TensorFlow backend.
W0930 15:35:33.988721 10056 deprecation_wrapper.py:119] From C:\Users\Public\Documents\Anaconda\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0930 15:35:34.026718 10056 deprecation_wrapper.py:119] From C:\Users\Public\Documents\Anaconda\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0930 15:35:34.031718 10056 deprecation_wrapper.py:119] From C:\Users\Public\Documents\Anaconda\lib\site-packages\keras\backend\tensorflow_backend.py:4115: The name tf.random_normal is deprecated. Please use tf.random.normal instead.

W0930 15:35:34.063717 10056 deprecation_wrapper.py:119] From C:\Users\Public\Documents\Anaconda\lib\site-packages\keras\backend\tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_

[[9.457703]] [[9.56]] [[0.9879933]]
[[12.127843]] [[8.87]] [[1.4160718]]
[[11.326981]] [[10.51]] [[1.0862705]]
[[10.038438]] [[15.]] [[0.6445872]]
[[11.57359]] [[7.99]] [[1.5156245]]
[[9.763948]] [[12.]] [[0.7959807]]
[[10.628305]] [[9.46]] [[1.1387535]]
[[8.78777]] [[8.48]] [[1.0413668]]
[[12.127843]] [[10.32]] [[1.1948106]]
[[12.086282]] [[15.33]] [[0.7730078]]
[[9.509942]] [[7.6]] [[1.2911496]]
[[8.988242]] [[7.84]] [[1.1688591]]
[[12.127843]] [[10.15]] [[1.2171068]]
[[9.602617]] [[8.16]] [[1.2026147]]
[[8.75536]] [[9.68]] [[0.8929814]]
[[12.127843]] [[14.]] [[0.85554343]]
[[9.173369]] [[12.65]] [[0.7005486]]
[[10.259942]] [[13.11]] [[0.76387256]]
[[12.127843]] [[16.]] [[0.74116594]]
[[11.208007]] [[13.]] [[0.85016775]]
[[9.911996]] [[9.]] [[1.1145724]]
[[9.178515]] [[9.61]] [[0.9496517]]
[[12.1293955]] [[15.]] [[0.7943693]]
[[10.056054]] [[13.84]] [[0.70437914]]
[[10.254719]] [[13.68]] [[0.72901255]]
[[9.326109]] [[10.3]] [[0.8948281]]
[[12.127843]] [[15.]] [[0.79425806]]
[[9.58882

C:\Users\Public\Documents\Anaconda\lib\site-packages\keras\callbacks.py:569: RuntimeWarning: Early stopping conditioned on metric `val_mean_absolute_error` which is not available. Available metrics are: val_loss,loss
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


[[10.732212]] [[9.56]] [[1.1375835]]
[[12.346267]] [[8.87]] [[1.4439676]]
[[11.240643]] [[10.51]] [[1.0771533]]
[[10.345486]] [[15.]] [[0.66658205]]
[[10.837312]] [[7.99]] [[1.409685]]
[[10.219028]] [[12.]] [[0.83750254]]
[[10.82147]] [[9.46]] [[1.1616948]]
[[5.471491]] [[8.48]] [[0.59563047]]
[[10.028466]] [[10.32]] [[0.9685847]]
[[12.61441]] [[15.33]] [[0.80996567]]
[[9.974388]] [[7.6]] [[1.3619494]]
[[8.204448]] [[7.84]] [[1.0535953]]
[[12.61441]] [[10.15]] [[1.270517]]
[[10.721239]] [[8.16]] [[1.3597245]]
[[10.324257]] [[9.68]] [[1.0745667]]
[[12.278207]] [[14.]] [[0.86714554]]
[[10.78706]] [[12.65]] [[0.83954]]
[[10.976737]] [[13.11]] [[0.82325906]]
[[12.61441]] [[16.]] [[0.77369046]]
[[10.37982]] [[13.]] [[0.78092134]]
[[10.212242]] [[9.]] [[1.1522917]]
[[10.419347]] [[9.61]] [[1.0944395]]
[[11.178335]] [[15.]] [[0.7262417]]
[[10.091415]] [[13.84]] [[0.7071418]]
[[10.475327]] [[13.68]] [[0.7464657]]
[[8.682713]] [[10.3]] [[0.8253469]]
[[12.479576]] [[15.]] [[0.81945384]]
[[10.726

C:\Users\Public\Documents\Anaconda\lib\site-packages\keras\callbacks.py:569: RuntimeWarning: Early stopping conditioned on metric `val_mean_absolute_error` which is not available. Available metrics are: val_loss,loss
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


...

C:\Users\Public\Documents\Anaconda\lib\site-packages\keras\callbacks.py:569: RuntimeWarning: Early stopping conditioned on metric `val_mean_absolute_error` which is not available. Available metrics are: val_loss,loss
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


[[11.007477]] [[9.56]] [[1.1698916]]
[[14.297276]] [[8.87]] [[1.6931386]]
[[10.057329]] [[10.51]] [[0.9521995]]
[[13.1182785]] [[15.]] [[0.8652062]]
[[9.33071]] [[7.99]] [[1.1929079]]
[[10.064284]] [[12.]] [[0.82338357]]
[[8.4162]] [[9.46]] [[0.8760332]]
[[7.9114423]] [[8.48]] [[0.923581]]
[[11.8332815]] [[10.32]] [[1.1630691]]
[[14.022523]] [[15.33]] [[0.908504]]
[[7.88305]] [[7.6]] [[1.0431479]]
[[8.066988]] [[7.84]] [[1.0333806]]
[[9.518023]] [[10.15]] [[0.9306281]]
[[9.900901]] [[8.16]] [[1.2445085]]
[[9.417911]] [[9.68]] [[0.9696654]]
[[11.522974]] [[14.]] [[0.80887145]]
[[11.622574]] [[12.65]] [[0.911505]]
[[12.822188]] [[13.11]] [[0.9761548]]
[[13.334963]] [[16.]] [[0.8218557]]
[[12.231585]] [[13.]] [[0.93575114]]
[[9.826768]] [[9.]] [[1.1038653]]
[[8.929131]] [[9.61]] [[0.92055196]]
[[12.579456]] [[15.]] [[0.8266086]]
[[11.911714]] [[13.84]] [[0.8493526]]
[[10.181855]] [[13.68]] [[0.72324806]]
[[9.636331]] [[10.3]] [[0.9283293]]
[[13.183143]] [[15.]] [[0.8698526]]
[[9.880867]] 

C:\Users\Public\Documents\Anaconda\lib\site-packages\keras\callbacks.py:569: RuntimeWarning: Early stopping conditioned on metric `val_mean_absolute_error` which is not available. Available metrics are: val_loss,loss
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


[[10.506729]] [[9.56]] [[1.1111183]]
[[11.897398]] [[8.87]] [[1.3866407]]
[[9.69496]] [[10.51]] [[0.9139345]]
[[11.813468]] [[15.]] [[0.77173835]]
[[7.2437444]] [[7.99]] [[0.8926251]]
[[8.994874]] [[12.]] [[0.7258097]]
[[8.996394]] [[9.46]] [[0.94494]]
[[9.822016]] [[8.48]] [[1.1803783]]
[[13.078997]] [[10.32]] [[1.2973057]]
[[15.256295]] [[15.33]] [[0.9948422]]
[[8.6576805]] [[7.6]] [[1.1612319]]
[[8.304052]] [[7.84]] [[1.068243]]
[[13.135597]] [[10.15]] [[1.3277274]]
[[8.723687]] [[8.16]] [[1.0791695]]
[[10.342143]] [[9.68]] [[1.0766368]]
[[13.8212595]] [[14.]] [[0.9862083]]
[[11.106999]] [[12.65]] [[0.8670972]]
[[10.657334]] [[13.11]] [[0.7967965]]
[[15.129788]] [[16.]] [[0.9418307]]
[[12.921871]] [[13.]] [[0.99346745]]
[[9.877702]] [[9.]] [[1.110264]]
[[8.783377]] [[9.61]] [[0.9035445]]
[[12.389151]] [[15.]] [[0.81297636]]
[[14.178754]] [[13.84]] [[1.026465]]
[[12.523196]] [[13.68]] [[0.90848064]]
[[10.645007]] [[10.3]] [[1.0372577]]
[[15.406207]] [[15.]] [[1.0290979]]
[[10.043782]

C:\Users\Public\Documents\Anaconda\lib\site-packages\keras\callbacks.py:569: RuntimeWarning: Early stopping conditioned on metric `val_mean_absolute_error` which is not available. Available metrics are: val_loss,loss
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning
